In [9]:
### First we handle our files ###
tree_infile = f"../000_000_341.tree"
alignment_infile = f"../000_000_341.fas.alg"
table_infile = f"../000_000_341.tsv"
uniprot_infile = f"../uniprot_2018_09.json"

### And we store them in different variables ###
try:
    with open(table_infile, "r") as table_file:
        table_info = table_file.readlines()
except:
    print ("No table infile.")
        
try:
    with open(alignment_infile, "r") as alignment_file:
        alignment_info = alignment_file.read()
except:     
    print ("No alignment infile.")
        
import json
uniprot_info = {}
with open(uniprot_infile, "r") as uniprot_file:
    for line in uniprot_file:
        uniprot_entry = json.loads(line)
        uniprot_info.update(uniprot_entry)

In [ ]:
print (uniprot_info)

In [10]:
### LOADING THE SPECIFIED TAG INFORMATION FROM OUR ALIGNMENT ###
def retrieve_features (feature_tag):
    uniprot_hit_hash = {}
    for line in table_info:
        hit_type = line.split("\t")[2]
        hit_name = line.split("\t")[1]
        if (hit_type == ("swissprot_best" or "swissprot_exact") and hit_name not in uniprot_hit_hash):
            for feature in uniprot_info[hit_name]["FT"]:
                if feature["ft"] == feature_tag:
                    if hit_name not in uniprot_hit_hash:
                        uniprot_hit_hash[hit_name] = [feature]
                    else:
                        uniprot_hit_hash[hit_name].append(feature)
    return uniprot_hit_hash

In [11]:
uniprot_hit_hash = retrieve_features("ACT_SITE")
print (uniprot_hit_hash)

{'O67077': [{'ft': 'ACT_SITE', 's': '419', 'e': '419', 'ann': '{ECO:0000255|HAMAP-Rule:MF_01458}.'}]}


In [12]:
### LOADING THE TREE WITH ETE3 AND PAINTING THE ACT SITES ###    
from ete3 import PhyloTree, TreeStyle
tree = PhyloTree(tree_infile, alignment=alignment_info, alg_format="fasta")
tree.show()